In [1]:
!pwd
import sys
sys.path.append('/workspace')

/workspace/notebooks


In [2]:
from src.core.db.config import DatabaseEnum
from src.core.db.models.pdf_models import Fincen8300Rev4
from src.core.db.session import DBContext, DbQuery, AppSession

[30/03/2021 04:20:58] [    INFO] [dotenv.main] [_get_stream():77] [PID:50 TID:140103025125184] -> Python-dotenv could not find configuration file /workspace/src/core/env/.local.env.local.


In [7]:
from src.core.db.models.main_models import ComplianceRunEvent
from src.core.db.models.pdf_models import IngestionEvent

def get_ingestion_event_and_write_to_compliance_run_event(ingestion_event_id: str):
    with DBContext(DatabaseEnum.PDF_INGESTION_DB) as pdf_db:
        result = pdf_db.query(IngestionEvent).filter(IngestionEvent.id == ingestion_event_id).one_or_none()
        if result is None:
            return None
    
        with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
            main_db.add(ComplianceRunEvent(id=ingestion_event_id,
                                           s3_bucket=result.s3_bucket,
                                           s3_key=result.s3_key,
                                           was_redacted = False,
                                           status = 'ok'
                                          ))
    return 'done'
        

    

In [8]:
r = get_ingestion_event_and_write_to_compliance_run_event('17d0e319-c247-432f-9d91-f4259430f25b')
print(r)
print(5)

[30/03/2021 04:23:34] [    INFO] [sqlalchemy.engine.base.Engine] [_begin_impl():730] [PID:50 TID:140103025125184] -> BEGIN (implicit)
[30/03/2021 04:23:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:50 TID:140103025125184] -> SELECT ingestion_event.id AS ingestion_event_id, ingestion_event.s3_bucket AS ingestion_event_s3_bucket, ingestion_event.s3_key AS ingestion_event_s3_key, ingestion_event.created_at AS ingestion_event_created_at, ingestion_event.updated_at AS ingestion_event_updated_at, ingestion_event.parsing_strategy_type_id AS ingestion_event_parsing_strategy_type_id 
FROM ingestion_event 
WHERE ingestion_event.id = %(id_1)s
[30/03/2021 04:23:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:50 TID:140103025125184] -> {'id_1': '17d0e319-c247-432f-9d91-f4259430f25b'}
[30/03/2021 04:23:34] [    INFO] [sqlalchemy.engine.base.Engine] [_begin_impl():730] [PID:50 TID:140103025125184] -> BEGIN (implicit)
[30/03/2021 04:23:34] 

done
5


In [ ]:
#db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
#result = db.execute("SELECT * from public.fincen8300_rev4")
from src.core.db.models.pdf_models import Fincen8300Rev4
app_pdf = AppSession(DatabaseEnum.PDF_INGESTION_DB)
session_pdf = app_pdf.instance
result = session_pdf.query(Fincen8300Rev4).all()
print(result)
print(len(result))

In [ ]:
from src.core.db.models.pdf_models import IngestionEvent
ingestion_event_id = "17d0e319-c247-432f-9d91-f4259430f25b"
#ingestion_event_id = "99d0e319-c247-432f-9d91-f4259430f25b"  # fake one
result = session_pdf.query(IngestionEvent).filter(IngestionEvent.id == ingestion_event_id).one_or_none()
if result is None:
    print("None found")
else:
    print(result.id)


# Get the fincen document corresponding to the ingestion id

In [ ]:
result = session_pdf.query(Fincen8300Rev4).filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id).one_or_none()
if result is None:
    print("None found")
else:
    print(result.id)
    print(type(result))


In [ ]:
from sqlalchemy.inspection import inspect
def convert_obj(obj):
    result = defaultdict(list)
    instance = inspect(obj)
    for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result
    
    
from collections import defaultdict
def query_to_dict(rset):
    result = defaultdict(list)
    for obj in rset:
        instance = inspect(obj)
        for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result
df = pd.DataFrame(convert_obj(result))
df.head()

# Read employee data into data source¶

In [ ]:
import pandas as pd
db = DbQuery(DatabaseEnum.MAIN_INGESTION_DB)
result = db.execute("SELECT * from public.employee")
df_employee = pd.DataFrame(result.fetchall())
df_employee.columns = result.keys()
df_employee.head(20)

In [ ]:
# only reasonable way to get into a dataframe
query = session_pdf.query(Fincen8300Rev4).filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id).statement
df = pd.read_sql(query, app_pdf._engine)
df.head()

# Map The Columns

In [ ]:
from src.sources.structured_data_source import StructuredDataSource
from src.mapping.pdfs.pdf_field_name_classifier import FieldNameClassifier
from src.mapping.pdfs.pdf_field_label_catalog import FieldLabelCatalog
from src.mapping.pdfs.pseudofield_generator import PseudofieldGenerator

pseudofield_generator = PseudofieldGenerator(fincen)
pseudofield_generator.generate()

def _create_column_relations_for(source, target):
    """Create column relations from canonical column identifiers."""
    gold_id_info = FieldNameClassifier.get_id_info_from_df(target.get_data())
    data_id_info = FieldNameClassifier.get_id_info_from_df(source.get_data())
    for identifier in FieldLabelCatalog:
        if identifier in gold_id_info and identifier in data_id_info:
            g_id = gold_id_info[identifier]
            d_id = data_id_info[identifier]
            source.create_column_relation(
                d_id.field_name, g_id.field_name, target
            )
            print(
                "New relation detected: %s" % str(source.column_relations[-1])
            )

In [ ]:
db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
result = db.execute("SELECT * from public.fincen8300_rev4 limit 10")
df = pd.DataFrame(result.fetchall())
df.columns = result.keys()
print(df.shape)